# Exploring the data

## Import Libraries & Load Data

In [1]:
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split

In [2]:
# Read data
labels = pd.read_csv('data/traininglabels.csv')
data = pd.read_csv('data/trainingdata.csv')


In [3]:
print(labels.head())
data.head()


      id    status_group
0  69572      functional
1   8776      functional
2  34310      functional
3  67743  non functional
4  19728      functional


,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [4]:
# Concat to make sole dataframe
wellsdf = pd.merge(labels, data, on='id')

## First look at the data

In [5]:
wellsdf.head()

,id,status_group,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,functional,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,functional,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,functional,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,non functional,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,functional,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [6]:
wellsdf.describe()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
count,59400.000000,59400.000000,59400.000000,59400.000000,5.940000e+04,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000
mean,37115.131768,317.650385,668.297239,34.077427,-5.706033e+00,0.474141,15.297003,5.629747,179.909983,1300.652475
std,21453.128371,2997.574558,693.116350,6.567432,2.946019e+00,12.236230,17.587406,9.633649,471.482176,951.620547
min,0.000000,0.000000,-90.000000,0.000000,-1.164944e+01,0.000000,1.000000,0.000000,0.000000,0.000000
25%,18519.750000,0.000000,0.000000,33.090347,-8.540621e+00,0.000000,5.000000,2.000000,0.000000,0.000000
50%,37061.500000,0.000000,369.000000,34.908743,-5.021597e+00,0.000000,12.000000,3.000000,25.000000,1986.000000
75%,55656.500000,20.000000,1319.250000,37.178387,-3.326156e+00,0.000000,17.000000,5.000000,215.000000,2004.000000
max,74247.000000,350000.000000,2770.000000,40.345193,-2.000000e-08,1776.000000,99.000000,80.000000,30500.000000,2013.000000


### Potential problems
- ID column not really a numerical column, probably should be dropped if unique identifier
- amount_tsh highly skewed, mostly zeros
- gps_height contains negative values, which may not make sense (check if any of Tanzania is below sea level)
- Longitude of zero is outside of Tanzania, probably put in place of missing value
- Unclear what num_private is, but 75%(+) of the values are zero, candidate to drop
- region_code and district_code are actually categorical
- 25%(+) wells being in an area with population seems unlikely
- Similarly, for construction year it is unlikely any wells in this database were actually built in 0 AD.  Probably used zero for missing values

In [7]:
# Examing data types and missing values
wellsdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 0 to 59399
Data columns (total 41 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   status_group           59400 non-null  object 
 2   amount_tsh             59400 non-null  float64
 3   date_recorded          59400 non-null  object 
 4   funder                 55765 non-null  object 
 5   gps_height             59400 non-null  int64  
 6   installer              55745 non-null  object 
 7   longitude              59400 non-null  float64
 8   latitude               59400 non-null  float64
 9   wpt_name               59400 non-null  object 
 10  num_private            59400 non-null  int64  
 11  basin                  59400 non-null  object 
 12  subvillage             59029 non-null  object 
 13  region                 59400 non-null  object 
 14  region_code            59400 non-null  int64  
 15  di

### Issues
- The features funder, installer, subvillage, public_meeting, scheme_management, and scheme_name all have explicit missing/NULL values.  Of these scheme_name may need to be dropped entirely due to how many values are missing, while the others can likely be imputed.
- date_record is an object, while construction year is an int

#### Check for Duplicates

In [8]:
wellsdf.duplicated().value_counts()

False    59400
dtype: int64

#### scheme_name

In [9]:
schemecounts = wellsdf.scheme_name.value_counts()
print(schemecounts)
print((schemecounts < 2).value_counts())

K                               682
None                            644
Borehole                        546
Chalinze wate                   405
M                               400
                               ... 
Ihanda spring box                 1
Malemeo gravity water supply      1
LANCH                             1
Embarway water scheme             1
Mradi wa maji wa Kiloleni         1
Name: scheme_name, Length: 2696, dtype: int64
False    1984
True      712
Name: scheme_name, dtype: int64


The second largest scheme is "None" (644), while other values look to be redundant with information from other features ("Borehole" is in sourcetype as well), and 712 only have one value.  Due to the confluence of these factors, it would be difficult and possibly counterproductive to impute values. Because the rows missing scheme_name represent nearly half the database, it is not practical to simply drop empty columns, so instead this feature will be dropped.

#### Checking region_code and district_code before switching them to categorical

In [10]:
wellsdf['region_code'].value_counts()


11    5300
17    5011
12    4639
3     4379
5     4040
18    3324
19    3047
2     3024
16    2816
10    2640
4     2513
1     2201
13    2093
14    1979
20    1969
15    1808
6     1609
21    1583
80    1238
60    1025
90     917
7      805
99     423
9      390
24     326
8      300
40       1
Name: region_code, dtype: int64

In [11]:
wellsdf['district_code'].value_counts()

1     12203
2     11173
3      9998
4      8999
5      4356
6      4074
7      3343
8      1043
30      995
33      874
53      745
43      505
13      391
23      293
63      195
62      109
60       63
0        23
80       12
67        6
Name: district_code, dtype: int64

#### Checking Longitude and Latitude 

In [12]:
wellsdf.latitude.value_counts()

-2.000000e-08    1812
-6.985842e+00       2
-3.797579e+00       2
-6.981884e+00       2
-7.104625e+00       2
                 ... 
-5.726001e+00       1
-9.646831e+00       1
-8.124530e+00       1
-2.535985e+00       1
-2.598965e+00       1
Name: latitude, Length: 57517, dtype: int64

In [13]:
wellsdf.longitude.value_counts()

0.000000     1812
37.540901       2
33.010510       2
39.093484       2
32.972719       2
             ... 
37.579803       1
33.196490       1
34.017119       1
33.788326       1
30.163579       1
Name: longitude, Length: 57516, dtype: int64

As there the exact same number of entries for -2.000000e-08 lat and 0 longitude it looks like that was used as the default value.

### Checking for class imbalance in target

In [14]:
wellsdf.status_group.value_counts()

functional                 32259
non functional             22824
functional needs repair     4317
Name: status_group, dtype: int64

The "functional needs repair" class is significantly smaller than the other two classes and may cause class imbalance issues.  Will need to SMOTE or convert to binary target based on business understanding.

## Data Cleaning

In [15]:
wells2 = wellsdf.copy()

# Triage.  In the interest of time, dropping all features with explicitly missing values
wells2 = wells2.drop(['funder', 'installer', 'scheme_name', 'public_meeting', 'scheme_management'], axis =1)

# Dropping extremely skewed features and id
wells2 = wells2.drop(['id', 'amount_tsh', 'num_private'], axis=1)


#### Create a year recorded column, drop date recorded

In [16]:
wells2.date_recorded = pd.to_datetime(wells2.date_recorded)
wells2['year_recorded'] = pd.DatetimeIndex(wells2['date_recorded']).year
wells2.drop('date_recorded', axis=1, inplace=True)

In [17]:
wells2.year_recorded.value_counts()

2011    28674
2013    24271
2012     6424
2004       30
2002        1
Name: year_recorded, dtype: int64

##### Construction year
Remove rows with zero, average the remainder, set zeros in wells2 df to the average construction year

In [18]:
con_year = wells2.drop(wells2[wells2.construction_year == 0].index) 
con_avg = con_year.construction_year.mean().round(0)

wells2.construction_year.replace(0, con_avg, inplace=True)

#### Convert Appropriate Numeric Columns to Categorical

In [19]:
wells2[['region_code', 'district_code', 'construction_year', 'year_recorded']] = wells2[['region_code', 'district_code', 'construction_year', 'year_recorded']].astype('str')

#### Lat & Long
The default value for longitude (0) is outside of Tanzania (40°29' E to 29°10' E), so it will be replaced by the mean longitude from the dataset.  While the default value for latitude is possible, it may skew our data more north (-2), so it will be replaced with the mean latitude.

Source: https://worldpopulationreview.com/countries/tanzania/location

In [20]:
longitude = wells2.drop(wells2[wells2.longitude == 0].index) 
long_avg = longitude.longitude.mean()
print(long_avg)

wells2.longitude.replace(0, long_avg, inplace=True)

35.149669123888835


In [21]:
latitude = wells2.drop(wells2[wells2.latitude == -2.000000e-08].index) 
lat_avg = latitude.latitude.mean()
print(lat_avg)

wells2.latitude.replace(-2.000000e-08, lat_avg, inplace=True)

-5.885572340514864


#### Convert target to binary
To meet the business understanding of wells that require an engineer's evaluation, "functional needs repair" and "non functional" will be combined into "needs work"

In [22]:
wells2.status_group = wells2.status_group.replace({'functional needs repair':'needs work',
                                                  'non functional':'needs work'})

In [23]:
wells2.status_group = wells2.status_group.map({'functional': 1, 'needs work': 0})

In [24]:
wells2.status_group.value_counts()

1    32259
0    27141
Name: status_group, dtype: int64

### Evaluate multicolinearity 

In [25]:
X = wells2.drop(['status_group'], axis=1)
y = wells2['status_group']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=52)

In [32]:
# Calculate VIF
vif = pd.DataFrame()
X_vif = X_train.select_dtypes(include=['float', 'int64'])
vif["features"] = X_vif.columns
vif["VIF"] = [variance_inflation_factor(X_vif.values, i) for i in range(len(X_vif.columns))]

vif                                                                

,features,VIF
0,gps_height,1.953814
1,longitude,7.089850
2,latitude,5.945180
3,population,1.170483


VIF for lat and longitude is above the normal threshold of 5, so may consider dropping them